# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [57]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [58]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,6.97,43,40,3.60,US,1682643083
1,1,kloulklubed,7.0419,134.2556,29.28,75,95,1.90,PW,1682643872
2,2,dudinka,69.4058,86.1778,-14.74,100,100,0.84,RU,1682643411
3,3,grytviken,-54.2811,-36.5092,1.35,82,17,3.62,GS,1682643375
4,4,banda aceh,5.5577,95.3222,27.41,76,100,1.61,ID,1682643872


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [59]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    s = "Humidity",
    c = "City",
    alpha = 0.5 
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [60]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_city_data_df = city_data_df.loc[city_data_df["Max Temp"] <= 26]
perfect_city_data_df = perfect_city_data_df.loc[perfect_city_data_df["Cloudiness"] == 0]
perfect_city_data_df = perfect_city_data_df.loc[perfect_city_data_df["Humidity"] <= 50]
perfect_city_data_df = perfect_city_data_df.loc[perfect_city_data_df["Wind Speed"] <= 5]

# Drop any rows with null values
perfect_city_data_df = perfect_city_data_df.dropna()

# Display sample data
perfect_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,pollock pines,38.7613,-120.5866,24.57,33,0,1.54,US,1682643880
83,83,ar ruways,24.1103,52.7306,25.47,42,0,2.90,AE,1682643890
113,113,sar-e pul,35.8333,66.1667,11.65,23,0,1.50,AF,1682643897
157,157,jamestown,42.0970,-79.2353,11.80,50,0,4.12,US,1682643276
169,169,namie,37.4833,141.0000,22.06,47,0,0.24,JP,1682643912


### Step 3: Create a new DataFrame called `hotel_df`.

In [61]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
32,pollock pines,US,38.7613,-120.5866,33,
83,ar ruways,AE,24.1103,52.7306,42,
113,sar-e pul,AF,35.8333,66.1667,23,
157,jamestown,US,42.0970,-79.2353,50,
169,namie,JP,37.4833,141.0000,47,
209,slave lake,CA,55.2834,-114.7690,32,
238,jiuquan,CN,39.7432,98.5174,25,
250,nurota,UZ,40.5614,65.6886,30,
252,saint-louis,SN,16.3333,-15.0000,38,
310,zayed city,AE,24.3768,54.5890,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [62]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter":'',
    "limit": 20,
    "bias":'',
    "apiKey":geoapify_key,
    "categories":'accommodation'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    longitude = hotel_df["Lng"][index]
    latitude = hotel_df["Lat"][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(url=base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pollock pines - nearest hotel: Westhaven Inn
ar ruways - nearest hotel: No hotel found
sar-e pul - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
namie - nearest hotel: HOTELなみえ
slave lake - nearest hotel: Lakeside Motor Inn
jiuquan - nearest hotel: 民族宾馆
nurota - nearest hotel: Ruslan Guesthouse & Tour
saint-louis - nearest hotel: No hotel found
zayed city - nearest hotel: Villa 80 adnoc
zhangjiakou shi xuanhua qu - nearest hotel: Century dynasty hotel
ra's bayrut - nearest hotel: ثري أو ناين
buka - nearest hotel: No hotel found
beyneu - nearest hotel: Railway Hostel
victorica - nearest hotel: Hotel Mazamet
gebeit - nearest hotel: No hotel found
idfu - nearest hotel: Hotel Diamond also known as Massa
panjab - nearest hotel: Bazar Dan Siah Darah - بازار دان سیاه دره


,City,Country,Lat,Lng,Humidity,Hotel Name
32,pollock pines,US,38.7613,-120.5866,33,Westhaven Inn
83,ar ruways,AE,24.1103,52.7306,42,No hotel found
113,sar-e pul,AF,35.8333,66.1667,23,No hotel found
157,jamestown,US,42.0970,-79.2353,50,DoubleTree Jamestown
169,namie,JP,37.4833,141.0000,47,HOTELなみえ
209,slave lake,CA,55.2834,-114.7690,32,Lakeside Motor Inn
238,jiuquan,CN,39.7432,98.5174,25,民族宾馆
250,nurota,UZ,40.5614,65.6886,30,Ruslan Guesthouse & Tour
252,saint-louis,SN,16.3333,-15.0000,38,No hotel found
310,zayed city,AE,24.3768,54.5890,44,Villa 80 adnoc


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [64]:
%%capture --no-display

# Configure the map plot
map_plot_2 = perfect_city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    s = "Humidity",
    c = "City",
    alpha = 0.5, 
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)